<a href="https://colab.research.google.com/github/LawTAGS/Deep-Learning-Classwork/blob/main/Assignments/DL2025_Tugas_8_10121091_Bryan_Guok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Identitas**#
Nama: Bryan Guok\
NIM : 10121091

#**Tugas 8:  Aplikasi Transformers**#
Dengan menggunakan program yang ada di Edunex  
1. Pilih sebuah film di web IMDB, ambil semua komentar dari film tersebut.
2. Gunakan pipeline summarization untuk membuat ringkasan setiap komentar.
3. Kemudian lakukan klasifikasi apakah komen tersebut positif atau negatif.
4. Hitung perbandingan dengan komentar positif terhadap total komentar yang anda
rangkum (Rasio = (jumlah komentar Positif / total komentar) × 100%).
5. Bandingkan dengan rating aslinya dengan web IMDB dan berikan analisis sederhana
terhadap perbandingannya.

##Library untuk Scraping Komentar YouTube##

In [ ]:
!pip install youtube-comment-downloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 6.6 MB/s eta 0:00:00


##**Analisis Sentimen**##

In [ ]:
from youtube_comment_downloader import YoutubeCommentDownloader
from transformers import pipeline
import re

# Fungsi untuk mengscrape komentar-komentar dari video trailer YouTube
def get_youtube_comments(video_url, max_comments=50):
    video_id = re.findall(r"v=([a-zA-Z0-9_-]+)", video_url)
    if not video_id:
        raise ValueError("URL tidak valid")
    video_id = video_id[0]

    downloader = YoutubeCommentDownloader()
    comments = []

    for comment in downloader.get_comments_from_url(f"https://www.youtube.com/watch?v={video_id}"):
        if comment['text'].strip():
            comments.append(comment['text'].strip())
        if len(comments) >= max_comments:
            break

    return comments


# Fungsi untuk menganalisis sentimen
def analyze_comments(comments):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
    sentiment_classifier = pipeline("sentiment-analysis")

    summarized_comments = []
    sentiment_results = []

    for comment in comments:
        try:
            if len(comment.split()) > 30:
                summary = summarizer(comment, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
            else:
                summary = comment
            summarized_comments.append(summary)

            sentiment = sentiment_classifier(summary)[0]
            sentiment_results.append({
                "original": comment,
                "summary": summary,
                "sentiment": sentiment['label'],
                "confidence": sentiment['score']
            })
        except Exception as e:
            print(f"Error: {e}")
            continue

    return sentiment_results


# Fungsi untuk menghitung rasio positif dari komentar
def analyze_ratio_and_compare(results, imdb_rating):
    total = len(results)
    positive = sum(1 for r in results if r['sentiment'] == 'POSITIVE')
    ratio = (positive / total) * 100 if total > 0 else 0

    print(f"\n Rasio komentar positif YouTube: {ratio:.2f}% ({positive}/{total})")
    print(f"IMDb Rating: {imdb_rating}/10")

    print("\n Analisis:")
    if ratio >= 75 and imdb_rating >= 8:
        print("Komentar YouTube dan rating IMDb sama-sama menunjukkan film ini sangat disukai.")
    elif ratio < 50 and imdb_rating >= 8:
        print("IMDb menunjukkan rating tinggi, tapi komentar YouTube lebih bervariasi atau negatif.")
    else:
        print("Komentar YouTube tidak sejalan dengan rating IMDb atau film kurang disukai.")

    return ratio


if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=PLl99DlL6b4"  # Trailer The Shawshank Redemption
    imdb_rating = 9.3  # IMDb rating The Shawshank Redemption

    comments = get_youtube_comments(video_url, max_comments=30)

    results = analyze_comments(comments)

    ratio = analyze_ratio_and_compare(results, imdb_rating)

    # Print hasil per komentar
    for r in results[:5]:  # Menampilkan 5 komentar pertama
        print("\n---")
        print(f"Komentar: {r['original']}")
        print(f"Ringkasan: {r['summary']}")
        print(f"Sentimen: {r['sentiment']} ({r['confidence']:.2f})")


Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



 Rasio komentar positif YouTube: 56.67% (17/30)
IMDb Rating: 9.3/10

 Analisis:
Komentar YouTube tidak sejalan dengan rating IMDb atau film kurang disukai.

---
Komentar: Saw this and “Tombstone” at theatres. I’m prob part of 5% of people who seen 2 of the greatest movies ever at theatres. 😊
Ringkasan: Saw this and “Tombstone” at theatres. I’m prob part of 5% of people who seen 2 of the greatest movies ever at theatres. 😊
Sentimen: POSITIVE (1.00)

---
Komentar: Hatlen should have took his bird  “Jake” with him. 😢
Ringkasan: Hatlen should have took his bird  “Jake” with him. 😢
Sentimen: NEGATIVE (1.00)

---
Komentar: Do thy get toguether romanticly? 🙏
Ringkasan: Do thy get toguether romanticly? 🙏
Sentimen: NEGATIVE (0.66)

---
Komentar: I hope this film will be show on ASTRO or TV3
Ringkasan: I hope this film will be show on ASTRO or TV3
Sentimen: NEGATIVE (0.98)

---
Komentar: ❤
Ringkasan: ❤
Sentimen: NEGATIVE (0.70)


##**Hasil**##

In [ ]:
total = len(results)
positive = sum(1 for r in results if r['sentiment'] == 'POSITIVE')
ratio = (positive / total) * 100
print(f"Rasio komentar positif YouTube: {ratio:.2f}%")

print(f"\nIMDb Rating: {imdb_rating}/10")

if ratio >= 75 and imdb_rating >= 8:
    print("Komentar YouTube dan rating IMDb sama-sama menunjukkan film ini disukai.")
elif ratio < 50 and imdb_rating >= 8:
    print("IMDb menunjukkan rating tinggi, tapi komentar YouTube tidak sesuai.")
else:
    print("Komentar YouTube tidak sesuai dengan rating IMDb.")

Rasio komentar positif YouTube: 56.67%

IMDb Rating: 9.3/10
Komentar YouTube tidak sesuai dengan rating IMDb.
